# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [1]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

### VectorDB Instance

In [3]:
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [4]:
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-3-small"
)

In [5]:
collection = chroma_client.get_or_create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)

### Add documents

In [12]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [9]:
# Semantic search: Finds most similar documents in the vector DB
query = "What is the best racing game?"
results = collection.query(
    query_texts=[query],
    n_results=5,
)

In [15]:
if not (metadata := results.get("metadatas", [[]])[0]):
    print("No results found.")
else:
    for meta in metadata:
        print(f"Name: {meta['Name']}")
        print(f"Platform: {meta['Platform']}")
        print(f"Genre: {meta['Genre']}")
        print(f"Publisher: {meta['Publisher']}")
        print(f"Description: {meta['Description']}")
        print(f"YearOfRelease: {meta['YearOfRelease']}")
        print("\n")

Name: Kinect Adventures!
Platform: Xbox 360
Genre: Party
Publisher: Microsoft Game Studios
Description: A collection of mini-games designed to showcase the capabilities of the Kinect motion sensor.
YearOfRelease: 2010


Name: Hades
Platform: Nintendo Switch
Genre: Roguelike
Publisher: Supergiant Games
Description: Zagreus attempts to escape the Underworld in this action roguelike.
YearOfRelease: 2020


Name: Dragon Age: Origins
Platform: PlayStation 3
Genre: Role-playing
Publisher: Electronic Arts
Description: A dark fantasy RPG in which players become a Grey Warden.
YearOfRelease: 2009


Name: Super Metroid
Platform: SNES
Genre: Action-adventure
Publisher: Nintendo
Description: A highly acclaimed Metroidvania classic starring Samus Aran.
YearOfRelease: 1994


Name: Metroid Prime
Platform: GameCube
Genre: Action-adventure
Publisher: Nintendo
Description: Samus investigates the mysteries of Tallon IV in a first-person adventure.
YearOfRelease: 2002


